In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

drive.mount('/content/drive')


train_path = "/content/drive/MyDrive/Contactless Human Activity Recogniton Using Deep Learning/Dataset Collection/sample images/5 classes/All layouts/train"
test_path =  "/content/drive/MyDrive/Contactless Human Activity Recogniton Using Deep Learning/Dataset Collection/sample images/5 classes/All layouts/test"


Mounted at /content/drive


In [ ]:
import timm
import time
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn import functional as F
import os
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
num_classes = 5
batch_size = 248
image_size = 224
num_epochs = 100
learning_rate = 1e-4

# Image transforms
test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.1)),  # zoom
    transforms.ColorJitter(brightness=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])




# Datasets
train_dataset = datasets.ImageFolder(train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(test_path, transform=test_transform)

# Dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=248,
    shuffle=True,
    num_workers=2,  # Try increasing this
    pin_memory=True
)


test_loader = DataLoader(
    test_dataset,
    batch_size=248  ,
    shuffle=False,         # Don't shuffle test data
    num_workers=2,         # Adjust based on Colab performance
    pin_memory=True        # Speeds up transfer to GPU
)
print(f"Total training samples: {len(train_dataset)}")
print(f"Total testing samples:  {len(test_dataset)}")


Total training samples: 36000
Total testing samples:  8999


In [ ]:
# Load pretrained  ViT variant
model = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=num_classes)
model = model.to(device)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy

# Early stopping setup
best_val_acc = 0
early_stop_counter = 0
patience = 4  # Stop if no improvement in 10 epochs
best_model_wts = None

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
train_acc_history = []
val_acc_history = []
train_loss_history = []
val_loss_history = []
metrics_log = []

start_time = time.time()

for epoch in range(num_epochs):
    epoch_start = time.time()

    model.train()
    total_loss, correct, total = 0, 0, 0
    y_true_train, y_pred_train = [], []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        y_true_train.extend(labels.cpu().numpy())
        y_pred_train.extend(preds.cpu().numpy())

    train_acc = correct / total
    train_prec, train_rec, train_f1, _ = precision_recall_fscore_support(
        y_true_train, y_pred_train, average='macro', zero_division=0)

    train_acc_history.append(train_acc)
    train_loss_history.append(total_loss)

    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0
    y_true_val, y_pred_val = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            val_loss += criterion(outputs, labels).item()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            y_true_val.extend(labels.cpu().numpy())
            y_pred_val.extend(preds.cpu().numpy())

    val_acc = val_correct / val_total
    val_acc_history.append(val_acc)

    val_prec, val_rec, val_f1, _ = precision_recall_fscore_support(
        y_true_val, y_pred_val, average='macro', zero_division=0)

    epoch_time = time.time() - epoch_start

    metrics_log.append({
        'epoch': epoch + 1,
        'train_acc': train_acc,
        'train_prec': train_prec,
        'train_rec': train_rec,
        'train_f1': train_f1,
        'val_acc': val_acc,
        'val_prec': val_prec,
        'val_rec': val_rec,
        'val_f1': val_f1,
        'epoch_time_sec': epoch_time
    })

    # Print epoch results
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Train Loss: {total_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Train F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%, Val F1: {val_f1:.4f}")
    print(f"Epoch Time: {epoch_time:.2f} sec\n")

    scheduler.step(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        best_model_wts = copy.deepcopy(model.state_dict())
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            break

total_time = time.time() - start_time

UnidentifiedImageError: Caught UnidentifiedImageError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torchvision/datasets/folder.py", line 245, in __getitem__
    sample = self.loader(path)
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torchvision/

In [ ]:
print("\n📊 Final Evaluation Report:")
for m in metrics_log:
    print(f"Epoch {m['epoch']:02}: "
          f"Train Acc={m['train_acc']:.4f}, Prec={m['train_prec']:.4f}, Rec={m['train_rec']:.4f}, F1={m['train_f1']:.4f} | "
          f"Val Acc={m['val_acc']:.4f}, Prec={m['val_prec']:.4f}, Rec={m['val_rec']:.4f}, F1={m['val_f1']:.4f} | "
          f"Time: {m['epoch_time_sec']:.2f}s")

print(f"\n✅ Best Epoch: {best_epoch}, Best Val Acc: {best_val_acc:.4f}")
print(f"⏱️ Total training time: {total_time:.2f} seconds")



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(train_acc_history, label='Train Accuracy')
plt.plot(val_acc_history, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import torch

# Collect all predictions and true labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Manually define the class names (in correct label order!)
class_names = ['highfive', 'handshake', 'kick left leg', 'punch right fist', 'push']

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Print classification report with renamed classes
print(classification_report(all_labels, all_preds, target_names=class_names))
